In [101]:
import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from torch import nn


In [102]:
data = pd.read_csv('train.csv')
data.head(10)

,id,sentence,label
0,fde9e435-8186-4cb3-8ec1-1be67ddb5f96,"""Трудно е класически оркестър и рок банда да с...",OBJ
1,bb522430-40f0-4781-9910-92a1aefd013b,"Следователно, Москва е пазителка на православн...",OBJ
2,d6a84f01-9153-4f3b-bca6-ed2b2edc6a9e,От Washington Post са изготвили подробен матер...,OBJ
3,3623488a-c528-4509-a92d-9ad4b49099ec,И пак така относно заслугите за постигнатото о...,OBJ
4,587b0e27-6ac8-433f-9b99-adf8d9c7c0a2,Понякога удобството да разтвориш набързо стран...,OBJ
5,f2d2aab0-25af-4bef-b678-badd5b390e8d,"Вчера Барак Обама, отиващият си тъжен стопанин...",SUBJ
6,f94840f8-7fb6-42e3-aa84-0a250d58af5b,Дали защото Първият черен президент на САЩ си ...,SUBJ
7,b30bd670-07fd-40ac-82d7-c67988f57cc3,"И като доказателство за това, гръмна и следващ...",SUBJ
8,512b57e5-9b65-43a8-8dec-01bef61a3ad6,"Последният път, когато Америка се обърна навът...",SUBJ
9,d6bad64f-59f2-491e-9d99-103d2748d647,"Шок, бомба, ужас!",SUBJ


In [103]:
data['sentence'] = data['sentence'].str.lower()
data['label'] = [1 if el == 'OBJ' else 0 for el in data['label']]
data.head(10)

,id,sentence,label
0,fde9e435-8186-4cb3-8ec1-1be67ddb5f96,"""трудно е класически оркестър и рок банда да с...",1
1,bb522430-40f0-4781-9910-92a1aefd013b,"следователно, москва е пазителка на православн...",1
2,d6a84f01-9153-4f3b-bca6-ed2b2edc6a9e,от washington post са изготвили подробен матер...,1
3,3623488a-c528-4509-a92d-9ad4b49099ec,и пак така относно заслугите за постигнатото о...,1
4,587b0e27-6ac8-433f-9b99-adf8d9c7c0a2,понякога удобството да разтвориш набързо стран...,1
5,f2d2aab0-25af-4bef-b678-badd5b390e8d,"вчера барак обама, отиващият си тъжен стопанин...",0
6,f94840f8-7fb6-42e3-aa84-0a250d58af5b,дали защото първият черен президент на сащ си ...,0
7,b30bd670-07fd-40ac-82d7-c67988f57cc3,"и като доказателство за това, гръмна и следващ...",0
8,512b57e5-9b65-43a8-8dec-01bef61a3ad6,"последният път, когато америка се обърна навът...",0
9,d6bad64f-59f2-491e-9d99-103d2748d647,"шок, бомба, ужас!",0


In [104]:
vectorizer = TfidfVectorizer(max_features=4000)
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['label'],test_size=0.2)

In [105]:
print(y_train)

680    0
82     0
293    1
345    1
119    1
      ..
339    1
380    0
707    1
796    1
271    0
Name: label, Length: 640, dtype: int64


In [106]:
print(y_train.values)

[0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 0 1 0 1
 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 1 0 0
 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 0 0
 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1
 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0
 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1
 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0
 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1
 1 1 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1
 1 1 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1
 1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1
 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1
 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 1 0 0 1 0 1 1 0 0 1
 1 0 0 0 1 0 0 1 0 0 1 1 

In [107]:
print(y_train.values.reshape(-1,1).astype(np.float32))

[[0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.

In [ ]:
X_train = vectorizer.fit_transform(X_train).toarray().astype(np.float32)
X_test = vectorizer.transform(X_test).toarray().astype(np.float32)
y_train = y_train.values.reshape(-1,1).astype(np.float32)
y_test = y_test.values.reshape(-1,1).astype(np.float32)

In [109]:
X_train.shape

(640, 4000)

In [110]:
type(y_train)

numpy.ndarray

In [ ]:
class SubjectivityDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y
    #assert sizes of X & y
  def __len__(self):
    return len(self.X) #640

  def __getitem__(self,idx):
    return self.X[idx], self.y[idx]

In [ ]:
train_dataset = SubjectivityDataset(X_train, y_train)
test_dataset = SubjectivityDataset(X_test, y_test)

In [ ]:
class LinearRegression(nn.Module):
  def __init__(self, features_size,num_classes):
    super(LinearRegression, self).__init__()
    self.linear = nn.Linear(features_size,num_classes)
    self.sigmod = nn.Sigmoid()
  def forward(self, x):
    output = self.linear(x)
    output = self.sigmoid(output)
    return output

In [113]:
model = LinearRegression(X_train.shape[1], 1)

In [ ]:
sgd = torch.optim.SGD(model.parameters(), lr=0.001)
bce = nn.BCELoss() # -> Why this loss function?

In [ ]:
EPOCHS=5
BATCH_SIZE=16
def train(model, optimizer, loss_fn, dataset):
  model.train()
  for epoch in range(EPOCHS+1): # -> Why are epochs necessary?
    cost = []
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True) # -> What is batch size?
    for X,y in tqdm(dataloader):
      optimizer.zero_grad() # -> What does this mean?
      loss = model(...)
      cost += loss_fn.detach().cpu().item()
      
      loss.backward()
      optimizer.step()
      #Run model for input X
      #Calculate error of results for X against real y values
      #Compute gradients for every parameter
      #Update paremeter values
    print ("Epoch: {:04d} cost={:.9f}".format(epoch, np.mean(cost)))

In [ ]:
train(model, sgd, bce, train_dataset)

100%|██████████| 40/40 [00:00<00:00, 525.94it/s]


Epoch: 0000 cost=0.693333745


100%|██████████| 40/40 [00:00<00:00, 514.42it/s]


Epoch: 0001 cost=0.693167508


100%|██████████| 40/40 [00:00<00:00, 336.58it/s]


...

Epoch: 0003 cost=0.692845821


100%|██████████| 40/40 [00:00<00:00, 290.54it/s]


Epoch: 0004 cost=0.692690492


100%|██████████| 40/40 [00:00<00:00, 234.53it/s]

Epoch: 0005 cost=0.692532659
